In [18]:
%load_ext autoreload

%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sqlite3
db_name = "stocks.sqlite"



In [3]:
connection=sqlite3.connect(database=db_name,check_same_thread=False)

In [ ]:
ls_tick=['','','','','']

In [22]:
from getdata import YahooFinance
from getdata import SQLrepo

In [35]:
ticker='IBM'
data=YahooFinance(ticker)

x=data.get_data()

repo=SQLrepo(connection)

repo.insert_table(table_name=ticker,records=x)



Data from Yahoo Finance API downloaded successfully
Data inserted into IBM successfully


{'records_inserted': True, 'records_count': 15447}

In [38]:
repo.read_table(table_name="IBM",limit=5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1962-01-02 00:00:00-05:00,1.609603,1.609603,1.591518,1.591518,407940,0.0,0.0
1962-01-03 00:00:00-05:00,1.591516,1.605428,1.591516,1.605428,305955,0.0,0.0
1962-01-04 00:00:00-05:00,1.605429,1.605429,1.588735,1.589430,274575,0.0,0.0
1962-01-05 00:00:00-05:00,1.587344,1.587344,1.555346,1.558129,384405,0.0,0.0
1962-01-08 00:00:00-05:00,1.556737,1.556737,1.516392,1.528913,572685,0.0,0.0


In [32]:
import pandas as pd

df_app = pd.read_sql_query(f"SELECT * FROM AAPL", connection, parse_dates=['Date'], index_col='Date')

In [33]:
df_app.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099722,0.100155,0.099722,0.099722,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094953,0.094953,0.094519,0.094519,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.088015,0.088015,0.087582,0.087582,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089749,0.090183,0.089749,0.089749,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.092351,0.092785,0.092351,0.092351,73449600,0.0,0.0


In [34]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10694 entries, 1980-12-12 00:00:00-05:00 to 2023-05-12 00:00:00-04:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          10694 non-null  float64
 1   High          10694 non-null  float64
 2   Low           10694 non-null  float64
 3   Close         10694 non-null  float64
 4   Volume        10694 non-null  int64  
 5   Dividends     10694 non-null  float64
 6   Stock Splits  10694 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 668.4+ KB
